In [1]:
from kinconnect_api.config import MONGO_CONNECTION_STRING
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_fireworks import FireworksEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from textwrap import dedent
# Connect to MongoDB
client = MongoClient(MONGO_CONNECTION_STRING)
db = client['kinconnect']
profiles_collection = db['demo_profiles']
vector_search_index = "demo_embedding_idx"


2024-06-14 19:36:39.274 | INFO     | kinconnect_api.config:<module>:15 - PROJ_ROOT path is: /Users/nehiljain/code/kinconnect/kinconnect_api


In [5]:


def get_career(item):
    return dedent(f'''
Company Name: {item['company']}
Title: {item['title']}
Description: {item['description']}
Start Date: {item['start_date']}
End Date: {item['end_date']}
''')

def get_project(item):
    return dedent(f'''
Title: {item['title']}
Description: {item['description']}
''')


def get_profile_doc(profile: dict):

    if 'name' not in profile:
        return None
    if 'past_projects' in profile and len(profile['past_projects']) > 0:
        past_projects_str = '\n'.join(get_project(item) for item in profile['past_projects'])
    else:
        past_projects_str = ""
    if 'career' in profile and len(profile['career']) > 0:
        career_str = '\n'.join(get_career(item) for item in profile['career'])
    else:
        career_str = ""

    prodile_doc = dedent(f'''

    This is the profile of {profile['name']}

    # Elevator Pitch of the person
    {profile['elevator_pitch']}

    # Career History and work experience of the person
    {career_str}

    # Past Projects and Portfolio
    {past_projects_str}

    # Skillset they have which will be useful for them
    {profile['skills']}

    # Their current focus and interests to match with relevant people
    {profile['interests']}


    # Awards and Honors they have won over their life
    {profile['honors']}
    ''')
    print(f"profile_doc: {profile}")
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(prodile_doc)
    
    print(f"inserting {len(md_header_splits)} documents")
    vector_search = MongoDBAtlasVectorSearch.from_documents(
        documents = md_header_splits,
        embedding = FireworksEmbeddings(model="nomic-ai/nomic-embed-text-v1.5"),
        collection = profiles_collection,
        index_name = vector_search_index
    )
    return vector_search



all_profiles = list(profiles_collection.find({}))
profile_docs = [get_profile_doc(profile) for profile in all_profiles]
 

profile_doc: {'_id': ObjectId('666cfd62e6c1d0ec5d80eed7'), 'name': 'Chloe', 'career': [{'company': 'Netflix', 'title': 'Senior Software Engineer', 'description': 'Led the development and deployment of an advanced recommendation engine for Netflix. This project aimed to enhance the accuracy and personalization of content recommendations for users by integrating machine learning algorithms. The system utilized user behavior data, viewing history, and ratings to predict and suggest content that matched user preferences. The project included a real-time processing pipeline to ensure recommendations were updated dynamically as user interactions occurred.', 'start_date': 'Jun 2018', 'end_date': 'Present'}], 'elevator_pitch': 'Experienced back end developer with 10 years of experience. Built and deployed backend databases for Netflix, including adding AI functionality to Netflix recommendation engines. Interested in Rags and AI LLMs.', 'honors': [], 'interests': ['Rags', 'AI LLMs'], 'past_pro

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(prodile_docs[0])



In [ ]:
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_fireworks import FireworksEmbeddings

# Create the vector store
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents = docs,
    embedding = FireworksEmbeddings(model="nomic-ai/nomic-embed-text-v1.5"),
    collection = profiles_collection,
    index_name = vector_search_index
)